In [6]:
!pip install tensorflow opencv-python numpy

In [7]:
import os
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau


In [8]:
data_dir = "Augmented"


In [9]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2,
    validation_split=0.2
)

val_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

In [10]:
train_data = train_datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

val_data = val_datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

Found 9578 images belonging to 6 classes.
Found 2392 images belonging to 6 classes.


In [11]:
# === Step 2: Improved model architecture ===
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Unfreeze last few layers for fine-tuning
for layer in base_model.layers[:-20]:
    layer.trainable = False
for layer in base_model.layers[-20:]:
    layer.trainable = True

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)  # Add dropout for regularization
predictions = Dense(train_data.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)


In [12]:
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True),
    ReduceLROnPlateau(factor=0.2, patience=3)
]

In [14]:
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=10,  # More epochs with early stopping
    callbacks=callbacks,
    verbose=1
)

Epoch 1/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 590s 2s/step - accuracy: 0.4820 - loss: 1.2877 - val_accuracy: 0.4356 - val_loss: 1.5406 - learning_rate: 1.0000e-04
Epoch 2/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 552s 2s/step - accuracy: 0.6206 - loss: 0.9926 - val_accuracy: 0.5096 - val_loss: 1.4105 - learning_rate: 1.0000e-04
Epoch 3/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 566s 2s/step - accuracy: 0.7017 - loss: 0.8039 - val_accuracy: 0.5786 - val_loss: 1.2488 - learning_rate: 1.0000e-04
Epoch 4/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 453s 2s/step - accuracy: 0.7584 - loss: 0.6675 - val_accuracy: 0.5957 - val_loss: 1.3685 - learning_rate: 1.0000e-04
Epoch 5/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 433s 1s/step - accuracy: 0.7914 - loss: 0.5704 - val_accuracy: 0.6246 - val_loss: 1.2321 - learning_rate: 1.0000e-04
Epoch 6/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 468s 2s/step - accuracy: 0.8223 - loss: 0.4947 - val_accuracy: 0.6635 - val_loss: 1.2578 - learning_rate: 1.0000e-04
Epoch 7/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 433s 1s/step - acc

In [15]:
train_loss, train_acc = model.evaluate(train_data)
val_loss, val_acc = model.evaluate(val_data)

print(f"Training Accuracy: {train_acc:.4f}")
print(f"Validation Accuracy: {val_acc:.4f}")
print(f"Difference: {abs(train_acc - val_acc):.4f}")

if abs(train_acc - val_acc) > 0.1:
    print("WARNING: Large gap between train/val accuracy suggests overfitting!")

model.save('skin_disease_model.h5')


300/300 ━━━━━━━━━━━━━━━━━━━━ 377s 1s/step - accuracy: 0.8879 - loss: 0.3078
75/75 ━━━━━━━━━━━━━━━━━━━━ 57s 754ms/step - accuracy: 0.7295 - loss: 1.0127
Training Accuracy: 0.8879
Validation Accuracy: 0.7295
Difference: 0.1584


In [18]:
def predict_skin_disease(image_path=None):
    model = load_model('skin_disease_model.h5')
    if image_path:
        img = cv2.imread(image_path)
        if img is None:
            print("Error: Could not load image")
            return
    else:
        cap = cv2.VideoCapture(0)
        ret, img = cap.read()
        cap.release()
        if not ret:
            print("Failed to capture image from camera.")
            return

    processed = preprocess_image(img)
    prediction = model.predict(processed)
    
    # Get confidence scores
    confidence = np.max(prediction)
    class_idx = np.argmax(prediction)
    class_name = list(train_data.class_indices.keys())[class_idx]
    
    print(f"Predicted: {class_name}")
    print(f"Confidence: {confidence:.4f}")
    
    # Show top 3 predictions
    top_3_idx = np.argsort(prediction[0])[-3:][::-1]
    print("\nTop 3 predictions:")
    for i, idx in enumerate(top_3_idx):
        class_name = list(train_data.class_indices.keys())[idx]
        conf = prediction[0][idx]
        print(f"{i+1}. {class_name}: {conf:.4f}")